In [4]:
# coding: utf-8

# # Importing Module
# - pandas for formatting or provides table
# - seabourn is wrapper over matplotlib

# In[1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#get_ipython().magic('matplotlib inline')

In [5]:
# In[4]:

df = pd.read_csv("before_after.csv",encoding= "utf-8")
df["Before"][0]


'face updated\nhad bypass in 2010. \ntakes following medications:\n1. metoprolol (blood pressure) - diagnosed 2014 - takes 25 mg daily \n2. levothyroxine (thyroid) - diagnosed 2016 - takes 25 mg daily \n'

In [6]:
from nltk.corpus import stopwords,wordnet as wn
from nltk.tokenize import wordpunct_tokenize,sent_tokenize
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import re


# In[18]:

In [7]:
def rem_punt(doc):
    ans = re.sub('"|\\n|\(|\)|,|\.|[$!--+@#:]',' ',doc)
    ans = re.sub(' +',' ',ans)
    ans = ans.lower()
    return ans


# In[19]:

df['RmNoise'] = df['Before'].apply(rem_punt)


# In[20]:

stop_word = set(stopwords.words('english'))


# In[21]:

def tokenize(document): 
    lemmy = []
    for sent in sent_tokenize(document):
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            #print(token,tag)
            if token in stop_word:
                 continue
            lemma = lemmatize(token, tag)
            lemmy.append(lemma)
    return lemmy

def lemmatize(token, tag):
    tag = {
          'N': wn.NOUN,
          'V': wn.VERB,
          'R': wn.ADV,
          'J': wn.ADJ
    }.get(tag[0], wn.NOUN)
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(token, tag)

In [8]:
# In[22]:

df['Lemmitize'] = df['RmNoise'].apply(tokenize)


df.to_csv('Script2data.csv',index=False, encoding = "utf-8")

In [9]:
# In[25]:

df = pd.read_csv('Script2data.csv')


# # Statistical Modeling 

# In[26]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer,LabelEncoder
from sklearn.metrics import accuracy_score,classification_report


# In[28]:

X = df['Lemmitize']
y = df['Offer']
#lab_y = LabelEncoder()
#y = lab_y.fit_transform(y)

In [37]:
X[0]

"['face', 'update', 'bypass', '2010', 'take', 'follow', 'medication', '1', 'metoprolol', 'blood', 'pressure', 'diagnose', '2014', 'take', '25', 'mg', 'daily', '2', 'levothyroxine', 'thyroid', 'diagnose', '2016', 'take', '25', 'mg', 'daily']"

In [38]:
# In[29]:

X_train,X_test,y_train,y_test = train_test_split(X,y)


# In[30]:

vect = TfidfVectorizer(max_df=0.8, max_features=15000, min_df=0.01, use_idf=True , ngram_range=(1,3))

In [76]:

#from xgboost.sklearn import XGBClassifier
#model1 = XGBClassifier(nthread=4,n_estimators=1000)


# Navie Bayes

from sklearn.naive_bayes import GaussianNB,MultinomialNB
model2 = GaussianNB()


# ExtraTree Classifier

from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
model3 = RandomForestClassifier(n_estimators=600,n_jobs=-1)

# SVM Classifier

from sklearn.svm import SVC
model4 = SVC()


# Logistic Regression 

from sklearn.linear_model import LinearRegression,SGDClassifier,LogisticRegression
model5 = LogisticRegression()

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model6 = LinearDiscriminantAnalysis()
'''
import matplotlib.pyplot as plt 
%matplotlib inline
plt.spy(vect)
'''

'\nimport matplotlib.pyplot as plt \n%matplotlib inline\nplt.spy(vect)\n'

In [77]:
# Model Fitting

import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

def model_making(vect , model , X_train , y_train , X_test , y_test):
    
    clf = make_pipeline(vect,model)
    clf.fit(X_train,y_train)
    pd = clf.predict(X_test)
    y_pred = clf.predict(X_test)
    print ("=====Accuracy Score ", "{0:.2f}".format(metrics.accuracy_score(y_test, y_pred)*100), "%")
    print ("=====Confusion Matrix")
    print (metrics.confusion_matrix(y_test, y_pred))
    target_names = ['class 0', 'class 1', 'class 2']
    print(classification_report(y_test, y_pred, target_names=target_names))
    

In [79]:
model_making(vect, model1, X_train, y_train, X_test, y_test)

=====Accuracy Score  100.00 %
=====Confusion Matrix
[[1]]
             precision    recall  f1-score   support

    class 0       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00         1



In [81]:
model_making(vect, model3, X_train, y_train, X_test, y_test)

=====Accuracy Score  100.00 %
=====Confusion Matrix
[[1]]
             precision    recall  f1-score   support

    class 0       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00         1



In [82]:
model_making(vect, model5, X_train, y_train, X_test, y_test)

=====Accuracy Score  100.00 %
=====Confusion Matrix
[[1]]
             precision    recall  f1-score   support

    class 0       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00         1



In [80]:
X_test


0    ['face', 'update', 'bypass', '2010', 'take', '...
Name: Lemmitize, dtype: object